# 라이브러리

In [34]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [35]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [36]:
unique(data$취업직종대분류)

[1] "경영·행정·사무직"                       
 [2] "보건·의료직"                            
 [3] "전기·전자 설치·정비·생산직"             
 [4] "사회복지·종교직"                        
 [5] "청소 및 기타 개인서비스직"              
 [6] "돌봄 서비스직(간병·육아)"               
 [7] "음식 서비스직"                          
 [8] "스포츠·레크리에이션직"                  
 [9] "경호·경비직"                            
[10] "운전·운송직"                            
[11] "제조 단순직"                            
[12] "예술·디자인·방송직"                     
[13] "인쇄·목재·공예 및 기타 설치·정비·생산직"
[14] "교육직"                                 
[15] "영업·판매직"                            
[16] "섬유·의복 생산직"                       
[17] "식품 가공·생산직"                       
[18] "기계 설치·정비·생산직"                  
[19] "건설·채굴직"                            
[20] "관리직(임원·부서장)"                    
[21] "미용·예식 서비스직"

In [67]:
# table 결과
table_result <- table(data$취업직종대분류)

# table 결과를 data.frame으로 변환
table_df <- as.data.frame(table_result)

# CSV 파일로 저장
write.csv(table_df, file = "data_취업직종대분류.csv", row.names = FALSE)


In [37]:
table(data$취업직종대분류)


                            건설·채굴직                        경영·행정·사무직 
                                    134                                    8961 
                            경호·경비직                     관리직(임원·부서장) 
                                    879                                     175 
                                 교육직                   기계 설치·정비·생산직 
                                    174                                     243 
               돌봄 서비스직(간병·육아)                      미용·예식 서비스직 
                                    452                                     266 
                            보건·의료직                         사회복지·종교직 
                                   1609                                     230 
                       섬유·의복 생산직                   스포츠·레크리에이션직 
                                    182                                     339 
                       식품 가공·생산직                             영업·판매직 
                                    276   

In [38]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [39]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [40]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [41]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [42]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [43]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12


In [44]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

# 변동사항

In [45]:
data_filtered_subset$target<-ifelse(data_filtered_subset$취업직종대분류 == "돌봄 서비스직(간병·육아)", 1, 0)


In [46]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [47]:
data_filtered_subset <- subset(data_filtered_subset, select = -취업직종대분류)

# 시험준비

In [48]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [49]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [50]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [51]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=100)

In [52]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [53]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [54]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =10)

# weights:  471
initial  value 10580.453566 
iter  10 value 1467.584094
iter  20 value 1328.883827
iter  30 value 1203.313216
iter  40 value 1085.507263
iter  50 value 1014.845227
iter  60 value 970.005144
iter  70 value 936.287189
iter  80 value 918.275144
iter  90 value 903.513850
iter 100 value 891.412780
final  value 891.412780 
stopped after 100 iterations


# 검증하기

In [55]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [56]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [57]:
# KNN 검증하기
mean(knnmodel==y) 

[1] 0.9807821

In [58]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [59]:
#  RF 예측력
mean(predrf == y)

[1] 0.9819832

In [60]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.9819832

In [61]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.9819832

In [62]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.9787802

# 오분류표

In [63]:
#  Random Forest 오분류표
table(predrf,y)

      y
predrf    0    1
     0 7358  135
     1    0    0

In [64]:
#  SVM 오분류표
table(predmodelsvm,y)

            y
predmodelsvm    0    1
           0 7358  135
           1    0    0

In [65]:
#  KNN 오분류표
table(y, knnmodel)

   knnmodel
y      0    1
  0 7345   13
  1  131    4

In [66]:
# nnet 오분류표
table(prednnet,y)

        y
prednnet    0    1
       0 7334  135
       1   24    0

## 결과는 X 못쓴다.